# 2.Repitan el paso 1 para distintos valores de frecuencias de modulación. Exploren un rango de valores entre 40 Hz y 50 kHz para medir la función de transferencia $T(\omega)=\frac{V_o(\omega)}{V_i(\omega)}$.
- ¿La función obtenida es la esperada?
- Discutan cualitativamente los resultados.
- Optimización barrido: para hacer un diagrama de bode, un barrido óptimo no da pasos equiespaciados ni toma el mismo tiempo de integración para todas la frecuencias. ¿Cómo optimizarían el barrido teniendo esto en cuenta? 

Arrancamos haciendo lo mismo que antes: Nos aseguramos de tener el puerto correcto; para eso importamos visa y le preguntamos cuales son los puertos disponibles, después vemos si el GPIB es el instrumento que queremos: 

In [ ]:
import visa

rm = visa.ResourceManager()

print(rm.list_resources()) #Preguntamos los puertos

#Vemos que devuelve el puerto "GPIB0::11::INSTR" que es el que queremos, pero aún así lo chequeamos

inst = rm.open_resource("GPIB0::11::INSTR") #Inicializamos comunicación con ese puerto
print(inst.query("*IDN?")) #Le preguntamos que instrumento esta conectado al puerto

#Debería devolver que es el lock-in SR830

Creamos el objeto del lock-in

In [ ]:
from lockin import SR830

config = {
          'lockin_addr': 'GPIB0::11::INSTR', 
          'medicion_modo' : 0, #Canal A single-ended
          'display_modo' : 'RT', #Nos muestra R y Tita
          'sens' : 12, #Sensibilidad 20mV
          'slope' : 0, #Orden del filtro 6dB/oct
          't_int' : 8, #Frecuencia de corte 10Hz 
          'ref_intern' : True, #Él mismo genera la referencia
          'ref_freq' : 80, #Con frecuencia 80Hz
          'ref_v' : 0.1, #Con amplitud 0.1V
          }

lock = SR830(config)


Ahora, la actividad nos pide hacer un barrido de frecuencias y ver la función transferencia en función de éstas. Para eso creamos un vector de frecuencias entre 40 y 50k equiespaciadas logarítmicamente y medimos para cada una de ellas

In [ ]:
import numpy as np
from time import sleep
import matplotlib.pyplot as plt

frecs = np.logspace(np.log10(40), np.log10(50000), 100, dtype=int) #Vector 
#equiespaciado logarítimcamente entre 40 y 50k
Rs = []
titas = []

#Seteamos los parámetros relevantes: sensibilidad, frecuencia de corte
#y orden del filtro
sens = 12
frec = 8
orden = 3
lock.setSensibility(sens) 
lock.setIntegrationTime(frec)
lockin.setFilterSlope(orden)
sleep(3)

#Medimos para cada frecuencia
for frec in frecs:
    lock.setFreqReferencia(frec)
    sleep(0.5)
    R, tita = lock.getMedicion("RT")
    Rs.append(R)
    titas.append(tita)

#Guardamos los datos
Rs = np.array(Rs)
titas = np.array(titas)
np.savetxt(f"barrido_R_s{sens}_f{frec}_o{orden}.txt", Rs, delimiter=",")
np.savetxt(f"barrido_titas_s{sens}_f{frec}_o{orden}.txt", titas, delimiter=",")


Visualizamos los datos

In [ ]:
plt.plot(frecs, Rs, '.r')
plt.xscale("log")
plt.show()